In [ ]:
%load_ext autoreload
%autoreload 2
from logging import getLogger
from lewidi_lib import enable_logging
import pandas as pd
import seaborn as sns

pd.set_option("display.max_colwidth", 100)
enable_logging()
logger = getLogger(__name__)
sns.set_context("talk")

In [ ]:
from lewidi_lib import assert_path_exists, preds_file

dataset = "MP"
file = preds_file(
    dataset=dataset,
    split="train",
    template="60",
    model_id="Qwen/Qwen3-32B",
    run_name="1000ex_10loops",
)
rdf = pd.read_parquet(assert_path_exists(file))

In [ ]:
from lewidi_lib import (
    assign_cols_perf_metrics_softlabel,
    compute_diversity_by_problem,
    join_dataset,
    keep_only_highest_diversity_problems,
    process_rdf,
)

rdf = process_rdf(rdf, response_contains_steps=True, discard_invalid_pred=True)
answer_diversity = compute_diversity_by_problem(rdf)
rdf = rdf.merge(answer_diversity, on="dataset_idx")
len(rdf)

In [ ]:
joint_df = join_dataset(rdf)
joint_df = assign_cols_perf_metrics_softlabel(joint_df)
joint_df_subset = keep_only_highest_diversity_problems(joint_df)

In [ ]:
# answer_diversity.query("diversity == 'Q5'")[["dataset_idx", "diversity"]].to_parquet("high_diversity_ids.parquet")

# """
# copy (
#     select rdf.*
#     from (select * from '../../1000ex_10loops/preds/responses.parquet') as rdf
#     join 'high_diversity_ids.parquet' as ids on rdf.dataset_idx = ids.dataset_idx
#     )
# to 'responses.parquet';
# """

# Load Ratings

In [ ]:
# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/MP/train/1000ex_10loops/judge/gemini-2.5-flash/responses.parquet"
# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/MP/train/1000ex_10loops/judge/Qwen/Qwen3-32B/set2/t23/1000ex_10loops/responses.parquet"
# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/CSC/train/1000ex_10loops/judge/gemini-2.5-flash/responses.parquet"
# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/CSC/train/1000ex_10loops/judge/Qwen/Qwen3-32B/set2/t23/1000ex_10loops/responses.parquet"
# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/CSC/train/1000ex_10loops/judge/deepseek-ai/DeepSeek-R1-0528-Qwen3-8B/set2/t23/1000ex_10loops/responses.parquet"

In [ ]:
from lewidi_lib import (
    assign_col_response_parsed,
    discard_na_response_rows,
    process_ratings,
)
import numpy as np
from prm800k import mapping

# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/CSC/train/1000ex_10loops/judge/gemini-2.5-flash/t24/responses.parquet"
# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/MP/train/1000ex_10loops/judge/Qwen/Qwen3-32B/set2/t24/1000ex_10loops_q5div/responses.parquet"
# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/MP/train/1000ex_10loops/judge/deepseek-ai/DeepSeek-R1-0528-Qwen3-8B/set2/t24/1000ex_10loops_q5div/responses.parquet"

# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/MP/train/1000ex_10loops/judge/gemini-2.5-flash/t24/responses.parquet"
# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/CSC/train/1000ex_10loops/judge/Qwen/Qwen3-32B/set2/t24/1000ex_10loops_q5div/responses.parquet"
# ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/CSC/train/1000ex_10loops/judge/deepseek-ai/DeepSeek-R1-0528-Qwen3-8B/set2/t24/1000ex_10loops_q5div/responses.parquet"

# judge = "gemini-2.5-flash"
judge = "Qwen/Qwen3-32B"
# judge = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"

judge_file = {
    "gemini-2.5-flash": "gemini-2.5-flash/t24/responses.parquet",
    "Qwen/Qwen3-32B": "Qwen/Qwen3-32B/set2/t24/1000ex_10loops_q5div/responses.parquet",
    "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B": "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B/set2/t24/1000ex_10loops_q5div/responses.parquet",
}
ratings_file = file.parent.parent / "judge" / judge_file[judge]

ratings = pd.read_parquet(ratings_file)
logger.info(
    "Loaded %d ratings for %d different dataset_idxs",
    len(ratings),
    ratings["dataset_idx"].nunique(),
)
ratings = discard_na_response_rows(ratings)
ratings = assign_col_response_parsed(ratings)
ratings = process_ratings(
    ratings, operation=np.mean, cat_mapping=mapping(ok=0.0, bad=0)
)
ratings = ratings[
    ["dataset_idx", "run_idx", "step_ratings", "score", "reasoning", "response_parsed"]
]
ratings.rename(columns={"reasoning": "judge_reasoning"}, inplace=True)

In [ ]:
joint_df_subset = joint_df_subset.merge(
    ratings, on=["dataset_idx", "run_idx"], how="left"
)
joint_df_subset = discard_na_response_rows(joint_df_subset, col="score")
logger.info(
    "We have score for %d examples of %d different dataset_idxs",
    len(joint_df_subset),
    joint_df_subset["dataset_idx"].nunique(),
)

In [ ]:
from lewidi_lib import compute_n_steps_equality

joint_df_subset = assign_col_response_parsed(joint_df_subset)
compute_n_steps_equality(joint_df_subset, step_source="response_parsed")

# BoN Loss Stats

In [ ]:
len(joint_df_subset)

In [ ]:
from lewidi_lib import bootstrap_avg

np.random.seed(0)
bootstrap_avg(
    [
        joint_df_subset.groupby("dataset_idx").sample(n=1)["ws_loss"].mean()
        for _ in range(100)
    ]
)

In [ ]:
from lewidi_lib import select_max_score_df

np.random.seed(0)
bootstrap_avg(
    [
        select_max_score_df(joint_df_subset.sample(frac=1.0))["ws_loss"].mean()
        for _ in range(20)
    ]
)

In [ ]:
oracle = joint_df_subset.loc[joint_df_subset.groupby("dataset_idx")["ws_loss"].idxmin()]
oracle[["ws_loss", "score"]].apply(bootstrap_avg)

# Problem-Level Correlation

In [ ]:
def corr(df):
    coeff = np.corrcoef(df["score"], df["ws_loss"])[0, 1]
    return coeff


corrs = (
    joint_df_subset.groupby("dataset_idx")[["score", "ws_loss"]].apply(corr).fillna(0)
)
bootstrap_avg(corrs)

# Individual Examples

In [ ]:
# max_loss = joint_df_subset.loc[joint_df_subset.groupby("dataset_idx")["ws_loss"].idxmax()]
# row = max_loss.iloc[0]
# print(row["response"])

# Aggregate Stats

In [ ]:
ax = sns.histplot(joint_df_subset, x="score")
ax.grid(alpha=0.5, axis="y")
ax.figure.set_size_inches(6, 3)

# How does performance change with the number of samples?

In [ ]:
from lewidi_lib import draw_bon_k_times

np.random.seed(0)
perf_vs_samples = pd.DataFrame({"n_samples": range(1, 11)})
perf_vs_samples["ws_loss"] = perf_vs_samples["n_samples"].apply(
    draw_bon_k_times, joint_df=joint_df_subset, k=100, performance_col="ws_loss"
)
perf_vs_samples = perf_vs_samples.explode("ws_loss")

In [ ]:
axs = sns.lineplot(data=perf_vs_samples, x="n_samples", y="ws_loss")
axs.grid(alpha=0.5)
axs.set_xlabel("Number of samples")
axs.set_ylabel("Wasserstein Distance")

In [ ]:
judge_alias = judge.replace("/", "_")
perf_vs_samples.assign(judge=judge, dataset=dataset).to_parquet(
    f"../notebook/tables/bon_samples_vs_perf_nlp/{judge_alias}_{dataset}.parquet"
)